In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample

In [3]:
train_data = np.loadtxt("./data/mnist_train.csv", 
                        delimiter=",")

In [4]:
class Model:
    def __init__(self):
        self.x = []        
        self.y = []
        self.error = []
        self._x = Input()
        self._y = Input()
        self.nodes = {}
        
    def sequential(self, data_train, data_test):
        self.x = data_train
        self.y = data_test
        
    def dense(self, n_hidden, input_shape):
        # w1
        self.nodes.update({ Input(): np.random.randn(input_shape, n_hidden) })
        # b1
        self.nodes.update({ Input(): np.zeros(n_hidden) })
        
        # w2
        self.nodes.update({ Input(): np.random.randn(n_hidden, 10) })
        # b2
        self.nodes.update({ Input(): np.zeros(10) })
        
        self.init()
        
    def init(self):
        nodes = [*self.nodes]
        t1 = Transfer(self._x, nodes[0], nodes[1])
        t2 = Transfer(t1, nodes[2], nodes[3])
        a = Activation(t2)
        c = Classifier(a)
        Error(self._y, c)
    
    def sgd(self, params, learning_rate = 1e-2):
        for t in params:
            partial = t.gradients[t]
            t.value -= learning_rate * partial
            
    def train(self, epochs, batch_size, steps_per_epoch):
        
        for i in range(epochs):
            for j in range(steps_per_epoch):
                data_train_batch, data_test_batch = resample(self.x, self.y, n_samples = batch_size)
                self._x.value = data_train_batch
                self._y.value = data_test_batch
                sorted_nodes = sort({**{self._x: self.x}, **{self._y: self.y}, **self.nodes})
                
                for n in sorted_nodes:
                    n.forward()

                for n in sorted_nodes[::-1]:
                    n.backward()
                self.sgd([*self.nodes])
            self.error.append(float("{:.3f}".format(sorted_nodes[-1].value/steps_per_epoch)))
        print(self.error)
            
    def test(self, data_train, data_test):
            self._x.value = data_train
            self._y.value = data_test
            sorted_nodes = sort({**{self._x: self.x}, **{self._y: self.y}, **self.nodes})
            for n in sorted_nodes:
                n.forward()
            print(float("{:.3f}".format(sorted_nodes[-1].value/steps_per_epoch)))

    def plot(self):
        plt.plot(np.arange(len(self.error)), self.error)
        plt.xlabel('Epochs')
        plt.ylabel('Error')
        plt.show()
        
        
class Node:
    def __init__(self, inputs = []):
        self.inputs = inputs
        self.outputs = []
        self.value = None
        self.gradients = {}
        for n in inputs:
            n.outputs.append(self)

    def forward(self):
        raise NotImplementedError

    def backward(self):
        raise NotImplementedError
        
    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__) 
        
class Input(Node):
    def __init__(self):
        Node.__init__(self)

    def forward(self):
        pass

    def backward(self):
        self.gradients = {self: 0}
        for n in self.outputs:
            self.gradients[self] = self.gradients[self] + n.gradients[self]
        pass
    
class Transfer(Node):
    def __init__(self, x, w, b):
        Node.__init__(self, [x, w, b])

    def forward(self):
        x = self.inputs[0].value
        w = self.inputs[1].value
        b = self.inputs[2].value
        self.value = np.dot(x, w) + b

    def backward(self):
        self.gradients = {n: np.zeros_like(n.value) for n in self.inputs}
        
        for n in self.outputs:
            # x
            self.gradients[self.inputs[0]] = self.gradients[self.inputs[0]] + np.dot(n.gradients[self], self.inputs[1].value.T)
            # w
            self.gradients[self.inputs[1]] = self.gradients[self.inputs[1]] + np.dot(self.inputs[0].value.T, n.gradients[self])
            # b
            self.gradients[self.inputs[2]] = self.gradients[self.inputs[2]] + np.sum(n.gradients[self], axis = 0, keepdims = False)
            
class Activation(Node):
    def __init__(self, node):
        Node.__init__(self, [node])

    def _sigmoid(self, z):
        return 1. / (1. + np.exp(-z))
    
    def forward(self):
        self.value = self._sigmoid(self.inputs[0].value)

    def backward(self):
        ## self.inputs == Tranfer Object
        self.gradients = {n: np.zeros_like(n.value) for n in self.inputs}
        for n in self.outputs:
            sigmoid = self.value
            self.gradients[self.inputs[0]] = self.gradients[self.inputs[0]] + sigmoid * (1 - sigmoid) * n.gradients[self]            

class Classifier(Node):
    def __init__(self, node):
        Node.__init__(self, [node])

    def _softmax(self, z):
        e = exp(z)
        return e / e.sum()
    
    def forward(self):
        self.value = self._softmax(self.inputs[0].value)

    def _softmax_grad(self, z):
        return self._softmax(z)*(1-self._softmax(z))
        
    def backward(self):
        ## self.inputs == Tranfer Object
        self.gradients = {n: np.zeros_like(n.value) for n in self.inputs}
        for n in self.outputs:
            softmax_grad = self._softmax_grad(self.value)
            print(self.inputs, n)
            self.gradients[self.inputs[0]] = np.dot(self.gradients[self.inputs[0]] + softmax_grad, n.gradients[self].T)
            
class Error(Node):
    def __init__(self, y, a):
        Node.__init__(self, [y, a])
   
    def forward(self):
        y = self.inputs[0].value
        a = self.inputs[1].value
        self.value = np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))
        self.m = y.shape[0]
        
    def backward(self):
        y = self.inputs[0].value
        a = self.inputs[1].value
        grad = y
        grad[range(self.m),a.argmax(axis=1)] -= 1
        self.gradients[self.inputs[0]] = grad/self.m
        self.gradients[self.inputs[1]] = grad/self.m
        

        
# using kahn's algorithm
def sort(data):
    #get the input class nodes
    inputs = [n for n in data.keys()]
    nodes = [n for n in inputs]
    queue = {}
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in queue:
            queue[n] = {'in': set(), 'out': set()}
        for m in n.outputs:
            if m not in queue:
                queue[m] = {'in': set(), 'out': set()}
            queue[n]['out'].add(m)
            queue[m]['in'].add(n)
            nodes.append(m)
    G = []
    TS = set(inputs)
    while len(TS) > 0:
        n = TS.pop()

        if isinstance(n, Input):
            n.value = data[n]

        G.append(n)
        for m in n.outputs:
            queue[n]['out'].remove(m)
            queue[m]['in'].remove(n)
            if len(queue[m]['in']) == 0:
                TS.add(m)
    return G

In [5]:
#test_data = np.loadtxt("./data/mnist_test.csv", delimiter=",") 
#_x = test_data[:,1:]
#_y = test_data[:,0]
#_x = (_x - np.mean(_x, axis = 0)) / np.std(_x, axis = 0)

x = train_data[:,1:]
y = train_data[:,0]
y = np.array(pd.get_dummies(y).values)

# Normalize X
x = x/255



In [ ]:
input_shape = x.shape[1]
n_hidden = 20
epochs = 1
m = train_data.shape[0]
batch_size = 10
steps_per_epoch = m // batch_size

model = Model()
model.sequential(x, y)
model.dense(n_hidden, input_shape)
model.train(epochs, batch_size, steps_per_epoch)
#model.test(_x, _y)
model.plot()
